In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load your data
df = pd.read_csv(
    r'C:\Users\Jouke\Documents\evedata-logger\output\market_data_with_names_merged.csv',
    parse_dates=['date']
)

item_names = sorted(df['type_name'].unique())

item_dropdown = widgets.Combobox(
    options=item_names,
    placeholder='Type or select an item',
    description='Item:',
    ensure_option=True,
    continuous_update=False
)

output = widgets.Output()
display(item_dropdown, output)


Combobox(value='', continuous_update=False, description='Item:', ensure_option=True, options=(' Tyrant Blue Sa…

Output()

In [2]:
def plot_price_vs_volume_enhanced(item):
    with output:
        clear_output(wait=True)
        if not item or item not in df['type_name'].values:
            print("Please select a valid item.")
            return
        item_df = df[df['type_name'] == item].sort_values('date').copy()
        print(f"Loaded {len(item_df)} days for: {item}")

        # Calculate price change direction
        item_df['price_change'] = item_df['average'].diff()
        colors = item_df['price_change'].apply(
            lambda x: 'green' if x > 0 else ('red' if x < 0 else 'gray')
        )

        # Mark top 5% volume days (spikes)
        threshold = item_df['volume'].quantile(0.95)
        outlier_days = item_df[item_df['volume'] >= threshold]

        plt.figure(figsize=(9, 6))
        plt.scatter(item_df['average'], item_df['volume'], c=colors, alpha=0.7, label='All days')
        plt.scatter(
            outlier_days['average'], outlier_days['volume'],
            edgecolors='black', facecolors='none', s=120, linewidths=1.5, label='Volume Spike'
        )
        plt.title(f"{item} — Price vs. Volume\n(Green=up day, Red=down day, Black ring=volume spike)")
        plt.xlabel("Average Price (ISK)")
        plt.ylabel("Daily Volume")
        plt.legend()
        plt.grid(True, linestyle=':')
        plt.tight_layout()
        plt.show()

widgets.interactive_output(plot_price_vs_volume_enhanced, {'item': item_dropdown})


Output()

In [3]:
from IPython.display import Markdown as md, display

def summarize_price_volume(item):
    if not item or item not in df['type_name'].values:
        display(md("**Please select a valid item.**"))
        return
    item_df = df[df['type_name'] == item].sort_values('date').copy()
    item_df['price_change'] = item_df['average'].diff()
    threshold = item_df['volume'].quantile(0.95)
    outlier_days = item_df[item_df['volume'] >= threshold]

    # How many up vs. down days in spikes?
    up_spikes = outlier_days['price_change'] > 0
    down_spikes = outlier_days['price_change'] < 0

    # Overall price-volume correlation
    corr = item_df['average'].corr(item_df['volume'])
    corr_desc = (
        "positive" if corr > 0.15 else
        "negative" if corr < -0.15 else
        "weak"
    )

    # Generate human-friendly insight
    insight = f"""
### Price vs. Volume Summary for **{item}**
- **Price/volume correlation:** *{corr_desc}* ({corr:.2f})
- **High volume spikes (top 5% days):** {len(outlier_days)}
    - **With price up:** {up_spikes.sum()}
    - **With price down:** {down_spikes.sum()}
- **Typical pattern:** {"High volume on up-days (bullish, confirms strong rallies)." if up_spikes.sum() > down_spikes.sum() else "High volume on down-days (bearish, strong sell pressure)." if down_spikes.sum() > up_spikes.sum() else "Volume spikes split evenly between up and down days."}
- **Interpretation:** {(
    "Large rallies tend to be supported by strong trading volume, suggesting robust demand."
    if up_spikes.sum() > down_spikes.sum() else
    "Major sell-offs coincide with heavy volume, indicating panic or aggressive selling."
    if down_spikes.sum() > up_spikes.sum() else
    "Volume spikes do not clearly confirm either buying or selling dominance—watch for other signals."
)}
"""
    display(md(insight))

# Run this after you select an item!
summarize_price_volume(item_dropdown.value)


**Please select a valid item.**